In [1]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconstruct
from scipy import linalg

n = 10
m = 2
p = 8
s = 2
dtsys_a = np.random.normal(3,3,(n,n))
eig_vals = linalg.eigvals(dtsys_a)
while(not all(np.isreal(eig_vals))):
    dtsys_a = np.random.normal(3,3,(n,n))
    eig_vals = linalg.eigvals(dtsys_a)
print(f'Ac eig_vals: {eig_vals}')

dtsys_b = np.random.normal(1,1,(n,m))
dtsys_c = np.random.normal(1,1,(p,n)) # if given one sensor redundancy
dtsys_d = np.zeros((p,m))


# define input output data
init_state1 = np.random.normal(2,2,(n,1))
init_state2 = 2*init_state1
# u_seq = np.array([[1,1],[1,1],[1,1],[0,0]])
# assume sensors 1,3,5 are under attack
sensor_initial_states = [init_state2,init_state2,init_state1,init_state1,
                            init_state1,init_state1,init_state1,init_state1]
u_seq, tilde_y_his, noise_level = SSProblem.generate_attack_measurement(dtsys_a, dtsys_b, dtsys_c, dtsys_d,sensor_initial_states,
                                                                        s = s,is_noisy = False, noise_level=0.0001,u_seq = None)

# construct a problem instance
ss_problem = SSProblem(dtsys_a, dtsys_b, dtsys_c, dtsys_d, tilde_y_his, 
                        attack_sensor_count=s,input_sequence=u_seq,measurement_noise_level= noise_level )
# print(f'A: \n {ss_problem.A},  \n B:\n  {ss_problem.B}, \n C: \n {ss_problem.C}, \n D:\n {ss_problem.D}')

# ssr_solution = SecureStateReconstruct(ss_problem,possible_comb=comb1)
ssr_solution = SecureStateReconstruct(ss_problem)

soi = SecureStateReconstruct.compute_sparse_observability(ss_problem.A,ss_problem.C)
# eigenvalue observability index
eoi = SecureStateReconstruct.compute_eigenvalue_observability(ss_problem.A,ss_problem.C)
print(f'The problem have a sparse observability index {soi}, eigenvalue observability index: {eoi}, attacked sensor count: {s}')
# eigenspace related property
unique_eigvals, generalized_eigenspace_list, am_list, gm_list = SecureStateReconstruct.compute_sys_eigenproperty(ss_problem.A)
# print(f'Eigen properties. . \n generalized_eigenspace_list: \n {generalized_eigenspace_list} ')
print(f'unique_eigvals: {unique_eigvals.T}, algebraic multiplicities: {am_list}, geometric multiplicities: {gm_list}')

# online operations 
# decompose ssr to sub_ssr problems
def to_test_code1():
    subprob_a, subprob_c, subprob_y = ssr_solution.generate_subssr_data()
    subspace_states_attacked_sensors_list = []
    for j in range(subprob_a.shape[2]):
        sub_a = subprob_a[:,:,j]
        sub_c = subprob_c
        sub_y_his = subprob_y[:,:,j]

        subproblem = SSProblem(sub_a, dtsys_b, sub_c, dtsys_d,sub_y_his, 
                                attack_sensor_count=s,is_sub_ssr=True)
        sub_solution = SecureStateReconstruct(subproblem)
        states, attacked_sensors_list = sub_solution.solve_initial_state_subssr(error_bound = 1,subspace = generalized_eigenspace_list[j])

        if states is not None:
            states = states.transpose() # to zip
            subspace_states_attacked_sensors = list(zip(states,attacked_sensors_list))
            subspace_states_attacked_sensors_list.append(subspace_states_attacked_sensors) # [..., [xj,[0,1,2]], ...]

    full_state_list,corresp_sensors = sub_solution.compose_subspace_states(subspace_states_attacked_sensors_list)
    # if full_state_list is not None:
    #     state_ind = 0
    #     for full_state in full_state_list:
    #         # print('--------------------------------------------------------------------')
    #         # print(f'Estimated state {state_ind} - initial state 1 is \n {full_state.reshape(-1,1)-init_state1}')
    #         # print(f'Estimated state {state_ind} - initial state 2 is \n {full_state.reshape(-1,1)-init_state2}')
    #         state_ind = state_ind +1

def to_test_code2():
    possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1)

Ac eig_vals: [ 29.21223136+0.j -12.4859549 +0.j   9.57502139+0.j  -6.45194994+0.j
   6.70348804+0.j   4.88990559+0.j   3.13359901+0.j  -3.86103586+0.j
  -2.29197066+0.j  -0.79511794+0.j]


NameError: name 'dtsys_a' is not defined

In [4]:
import timeit

execution_time = timeit.timeit(to_test_code1, number=1000)

print(f'execution time {execution_time}s')

execution_time = timeit.timeit(to_test_code2, number=1000)

print(f'execution time {execution_time}s')

execution time 4.93262275699999s
execution time 0.8606646839998575s


In [4]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconstruct
from scipy import linalg
import numpy as np

np.random.seed(100)
TS = 0.02
s  = 0
Ac = np.matrix(
                [
                    # [0, 0, 1, 0],
                    # [0, 0, 0, 1],
                    # [0, 0, 0, 0],
                    # [0, 0, 0, 0]
                    [0, 1],  # x xdot
                    [0, 0],
                ])
Bc = np.matrix(
                [
                    # [0, 0],
                    # [0, 0],
                    # [1, 0],
                    # [0, 1]
                    [0],
                    [1],
                ]
            )
Cc = np.matrix(
                [
                    # [1, 0, 0, 0],
                    # [1, 0, 0, 0],
                    # [0, 1, 0, 0],
                    # [0, 1, 0, 0],
                    # [0, 0, 1, 0],
                    # [0, 0, 1, 0],
                    # [0, 0, 0, 1],
                    # [0, 0, 0, 1],
                    [1, 0],  # x1 xdot1 x2 xdot2
                    [0, 1],
                    [1, 0],
                    [0, 1],
                ]
            )

Dc = np.matrix(
                [
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0]
                    [0],
                    [0],
                    [0],
                    [0],
                ]
            )

dtsys_a, dtsys_b, dtsys_c, dtsys_d = SSProblem.convert_ct_to_dt(Ac,Bc,Cc,Dc,TS)
# define input output data
init_state1 = np.array([[1.],[2.]])
# u_seq = np.array([[1,1],[1,1],[1,1],[0,0]])
# assume sensors 1,3,5 are under attack
sensor_initial_states = [init_state1,init_state1,init_state1,init_state1]
# u_seq, tilde_y_his, noise_level = SSProblem.generate_attack_measurement(dtsys_a, dtsys_b, dtsys_c, dtsys_d,sensor_initial_states,
#                                                                         s = s,is_noisy = False, noise_level=0.001,u_seq = None)
u_seq = np.random.normal(3,2.5,size=(2,1))
tilde_y_his = np.random.normal(10,4,size=(2,4))

noise_level=0.001
ss_problem = SSProblem(dtsys_a, dtsys_b, dtsys_c, dtsys_d, tilde_y_his, 
                        attack_sensor_count=s,input_sequence=u_seq,measurement_noise_level= noise_level )
ssr_solution = SecureStateReconstruct(ss_problem)
possible_initial_states,corresp_sensors, _ = ssr_solution.solve_initial_state(error_bound = 1000)

possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1000)
print(f'u_seq:{u_seq}')
print(f'tilde_y_his: {tilde_y_his}')
print(f'possible_initial states: {possible_initial_states}')
print(f'possible states: {possible_states}')



residual min is [44.16147998]
residual min is [44.16147998]
x_new: [[12.26028815]
 [ 9.41102758]]
x_new: [[12.44928004]
 [ 9.4881616 ]]
ind:0
curr_state: [[12.44928004]
 [ 9.4881616 ]]
u_seq:[[-1.37441368]
 [ 3.85670101]]
tilde_y_his: [[14.61214321  8.99025585 13.92528315 12.05687537]
 [10.88471868  5.71982668  9.24201668 11.02000578]]
possible_initial states: [[12.07179271]
 [ 9.43851585]]
possible states: [[12.44928004]
 [ 9.4881616 ]]


In [5]:
x_old = possible_initial_states[:,0:1]
print(x_old)
x_new = SSProblem.update_state_one_step(dtsys_a, dtsys_b,x_old,u_seq[0,:])
print(x_new)

[[12.07179271]
 [ 9.43851585]]
[[12.26028815]
 [ 9.41102758]]


In [1]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconstruct
from scipy import linalg
import control as ct

# def to_test_code():
# randomize system to test 
init_state1 = np.array([[1.],[1.],[1.],[1.]])
dtsys_a = np.array( [[ 7.09221296,  4.50145174,  6.45752449,  3.09727833],
 [ 6.9494097,   3.62746914,  3.57307166, -0.82619306],
 [11.04056775,  3.08802872,  3.86302024,  5.47661976],
 [ 5.27031611,  5.20731065,  2.81388349, -4.12382917]])
dtsys_c = np.random.normal(1,1,size=[8,4]) # if given enough sensor redundancy

eig_vals,eig_vecs = linalg.eig(dtsys_a)
I0 = np.eye(dtsys_a.shape[0])
# while(not all(np.isreal(eig_vals))):
#     dtsys_a = np.random.normal(3,3,size=[4,4])
#     eig_vals = linalg.eigvals(dtsys_a)
print(f'eig_vals: {eig_vals}')

eigenspaces = [eig_vecs[:,i:i+1] for i in range(eig_vecs.shape[1])]
r = len(eigenspaces)
# print(eigenspaces)
# print(f'test eigenvector: {linalg.norm((dtsys_a- eig_vals[0]*I0)@eigenspaces[0])}')

projections = []
x_components = []
for j in range(r):
    proj = SecureStateReconstruct.construct_proj(eigenspaces,j)
    xcomponent = proj@init_state1
    x_components.append(xcomponent)
    projections.append(proj)

x_recover  = np.sum(x_components,axis=0)
print(f'x:{init_state1}, x components: {x_components}, recovered: {x_recover}')


eig_vals: [18.73635446+0.j -1.26587636+0.j -3.16087944+0.j -3.85072548+0.j]
x:[[1.]
 [1.]
 [1.]
 [1.]], x components: [array([[1.11255015],
       [0.76321664],
       [1.1970287 ],
       [0.57768987]]), array([[ 0.02089954],
       [-0.06326829],
       [ 0.03658282],
       [-0.04071819]]), array([[-0.2668541 ],
       [ 0.14378073],
       [ 0.27110101],
       [ 0.10919792]]), array([[ 0.13340442],
       [ 0.15627091],
       [-0.50471253],
       [ 0.3538304 ]])], recovered: [[1.]
 [1.]
 [1.]
 [1.]]


In [4]:
for i in range(dtsys_c.shape[0]):
    obsrv_spaces_i = []
    obsrv_projs_i = []
    Ci = dtsys_c[i:i+1,:]
    obsrv_matrix = ct.obsv(dtsys_a,Ci)
    Yi = obsrv_matrix@init_state1
    for j in range(r):
        obsrv_space_ij = obsrv_matrix@eigenspaces[j]
        obsrv_spaces_i.append(obsrv_space_ij)
    
    Y_i_list = []
    for j in range(r):
        proj_obsrv_ij = SecureStateReconstruct.construct_proj(obsrv_spaces_i,j)
        Y_ij = proj_obsrv_ij@Yi
        obsrv_projs_i.append(proj_obsrv_ij)
        Y_i_list.append(Y_ij)
    Y_ij_sum = np.sum(Y_i_list,axis= 0)
    print(f'{i}th sensor, Yi - Y_ij_sum is {linalg.norm(Yi- Y_ij_sum)}')

    # solve for state for subssr problems
    x_recovered_list = []
    for j in range(r):
        Aj = projections[j]@dtsys_a
        observ_matrix = ct.obsv(Aj,Ci)
        # x_j = linalg.pinv(observ_matrix)@Y_i_list[j]
        x_j, residuals = SecureStateReconstruct.solve_lstsq_from_subspace(observ_matrix,Y_i_list[j],eigenspaces[j]) # rank defficient, not accurate
        print(f'residuals: {residuals}')
        x_j = x_j.reshape(-1,1)
        x_recovered_list.append(x_j)

        # print(f'j = {j}, the difference btw x_components[j] and x_j is {linalg.norm(x_components[j]- x_j)}')
        err = linalg.norm(observ_matrix@x_components[j] - Y_i_list[j])
        # print(f'x_j {x_j.T}, x_components[j] {x_components[j].T}')
        # print(f'j = {j}, the error bound is {err}')
    print(f'The sum of recovered states is {np.sum(x_recovered_list,axis=0).T}')


    


0th sensor, Yi - Y_ij_sum is 3.035084092208749e-08
residuals: 1.0474639173579756e-20
residuals: 2.7076536663913674e-19
residuals: 4.6031715152075815e-17
residuals: 4.198698980296493e-18
The sum of recovered states is [[1.00000001 1.         0.99999999 1.        ]]
1th sensor, Yi - Y_ij_sum is 7.203973825569127e-08
residuals: 8.535393616389226e-21
residuals: 1.3689056131808267e-17
residuals: 9.531615852850277e-17
residuals: 2.20203475009586e-16
The sum of recovered states is [[0.99999999 1.00000001 1.00000001 1.        ]]
2th sensor, Yi - Y_ij_sum is 3.4222414640897666e-08
residuals: 6.167017554042498e-21
residuals: 2.952083353712113e-19
residuals: 1.794502622380283e-16
residuals: 1.782676530746708e-16
The sum of recovered states is [[1. 1. 1. 1.]]
3th sensor, Yi - Y_ij_sum is 5.86790080231231e-08
residuals: 7.207035887179908e-21
residuals: 8.899318626667027e-18
residuals: 7.616066079443117e-16
residuals: 3.072353905777428e-17
The sum of recovered states is [[1. 1. 1. 1.]]
4th sensor, Y

In [4]:
import timeit

execution_time = timeit.timeit(to_test_code, number=1000)

print(f'execution time {execution_time}s')

eigenspace: [[-0.1877121 -0.j]
 [-0.20838112-0.j]
 [ 0.02652552-0.j]
 [ 0.95949876-0.j]]
eigenspace: [[-0.14395162-0.j]
 [ 0.80504661-0.j]
 [-0.54587424-0.j]
 [-0.18220651-0.j]]
eigenspace: [[ 0.53912497-0.j        ]
 [-0.61286176-0.30290382j]
 [ 0.3664726 +0.14658077j]
 [ 0.29319708+0.0155366j ]]
eigenspace: [[ 0.53912497-0.j        ]
 [-0.61286176+0.30290382j]
 [ 0.3664726 -0.14658077j]
 [ 0.29319708-0.0155366j ]]
The problem have a sparse observability index 7, eigenvalue observability index: 7
eigenspace: [[-0.1877121 -0.j]
 [-0.20838112-0.j]
 [ 0.02652552-0.j]
 [ 0.95949876-0.j]]
eigenspace: [[-0.14395162-0.j]
 [ 0.80504661-0.j]
 [-0.54587424-0.j]
 [-0.18220651-0.j]]
eigenspace: [[ 0.53912497-0.j        ]
 [-0.61286176-0.30290382j]
 [ 0.3664726 +0.14658077j]
 [ 0.29319708+0.0155366j ]]
eigenspace: [[ 0.53912497-0.j        ]
 [-0.61286176+0.30290382j]
 [ 0.3664726 -0.14658077j]
 [ 0.29319708-0.0155366j ]]
Eigen properties. unique_eigvals: [-5.49758228+0.j          2.63274871+0.j   

In [9]:
from scipy import linalg
Ac = np.random.normal(3,3,size=[4,4])
eig_vals = linalg.eigvals(Ac)
i = 0
while(not all(np.isreal(eig_vals))):
    Ac = np.random.normal(3,3,size=[4,4])
    eig_vals = linalg.eigvals(Ac)
    i = i+1
    print(i)

print(f'eig_vals: {eig_vals}')

eig_vals: [14.33994468+0.j -4.65799276+0.j  3.37893113+0.j  5.30334343+0.j]


In [4]:
dtsys_a = np.array( [[ 7.09221296,  4.50145174,  6.45752449,  3.09727833],
 [ 6.9494097,   3.62746914,  3.57307166, -0.82619306],
 [11.04056775,  3.08802872,  3.86302024,  5.47661976],
 [ 5.27031611,  5.20731065,  2.81388349, -4.12382917]])

unique_eigvals, generalized_eigenspace_list, am_list, gm_list = SecureStateReconstruct.compute_sys_eigenproperty(dtsys_a)
print(f'gm_list: {gm_list}')
print(f'unique_eigvals:{unique_eigvals}')
print(f'generalized_eigenspace_list:{generalized_eigenspace_list}')

eigenspace: [[ 0.20532311-0.j]
 [ 0.240517  -0.j]
 [-0.77680445-0.j]
 [ 0.54458134-0.j]]
eigenspace: [[-0.63374519-0.j]
 [ 0.3414613 -0.j]
 [ 0.64383105-0.j]
 [ 0.25933144-0.j]]
eigenspace: [[-0.24236437-0.j]
 [ 0.73369935-0.j]
 [-0.4242377 -0.j]
 [ 0.47219398-0.j]]
eigenspace: []
gm_list: [1, 1, 1, 0]
unique_eigvals:[-3.85072548+0.j -3.16087944+0.j -1.26587636+0.j 18.73635446+0.j]
generalized_eigenspace_list:[array([[ 0.20532311-0.j],
       [ 0.240517  -0.j],
       [-0.77680445-0.j],
       [ 0.54458134-0.j]]), array([[-0.63374519-0.j],
       [ 0.3414613 -0.j],
       [ 0.64383105-0.j],
       [ 0.25933144-0.j]]), array([[-0.24236437-0.j],
       [ 0.73369935-0.j],
       [-0.4242377 -0.j],
       [ 0.47219398-0.j]]), array([], shape=(4, 0), dtype=complex128)]


In [9]:
from scipy import linalg
eigval, eigvec = linalg.eig(dtsys_a)


nullspace:[]


In [16]:
a = np.array(range(24))
a = a.reshape(-1,2)
print(f'a before unvstack: {a}')
io_len = 4
a_prime = SecureStateReconstruct.unvstack(a,io_len)

print(f'a after unvstack: {a_prime[:,:,2]}')

a_colum_list = [a_prime[:,i:i+1] for i in range(a_prime.shape[1])]
a_v = np.vstack(a_colum_list)
print(f'a after unvstack then vstack: {a_v}')


a before unvstack: [[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]]
a after unvstack: [[16 17]
 [18 19]
 [20 21]
 [22 23]]
a after unvstack then vstack: [[[ 0  8 16]]

 [[ 2 10 18]]

 [[ 4 12 20]]

 [[ 6 14 22]]

 [[ 1  9 17]]

 [[ 3 11 19]]

 [[ 5 13 21]]

 [[ 7 15 23]]]


In [22]:
dtsys_a = np.random.normal(3,3,size=[4,4])
b = dtsys_a@np.array([[1],[1],[1],[1]])
b.shape

(4, 1)

In [ ]:
I = np.eye(dtsys_a.shape[0])
nullspace = linalg.null_space(dtsys_a - eigval[0]*I)
linalg.norm((dtsys_a - eigval[0]*I)@eigvec[:,0])
print(f'nullspace:{nullspace}')

In [2]:
import numpy as np
from mpmath import mp, mpf, matrix, eig

# Set precision (e.g., 50 decimal places)
mp.dps = 50

def compute_high_precision_eigenspaces(matrix_A):
    # Convert matrix to mpmath matrix with higher precision
    mp_matrix = matrix([list(map(mpf, row)) for row in matrix_A])
    
    # Compute eigenvalues and eigenvectors using mpmath
    eigenvalues, eigenvectors = eig(mp_matrix)
    
    eigenspaces = {}
    
    for i, eigenvalue in enumerate(eigenvalues):
        eigenvector = eigenvectors[:, i]
        
        # Normalize the eigenvector
        norm = mp.sqrt(sum(x**2 for x in eigenvector))
        eigenvector = eigenvector / norm
        
        # Convert back to numpy arrays for convenience
        eigenspaces[float(eigenvalue)] = np.array([float(x) for x in eigenvector])
    
    return eigenspaces

# Example matrix
A = np.array([[4, -2],
              [1, 1]], dtype=float)

eigenspaces = compute_high_precision_eigenspaces(A)

for eigenvalue, eigenvector in eigenspaces.items():
    print(f"Eigenvalue: {eigenvalue}")
    print(f"Eigenvector: {eigenvector}\n")


Eigenvalue: 3.0
Eigenvector: [0.89442719 0.4472136 ]

Eigenvalue: 2.0
Eigenvector: [0.70710678 0.70710678]



In [13]:
import numpy as np
from scipy import linalg

dtsys_a = np.array( [[ 7.09221296,  4.50145174,  6.45752449,  3.09727833],
 [ 6.9494097,   3.62746914,  3.57307166, -0.82619306],
 [11.04056775,  3.08802872,  3.86302024,  5.47661976],
 [ 5.27031611,  5.20731065,  2.81388349, -4.12382917]])

eigval, eigvec = linalg.eig(dtsys_a)
I = np.eye(dtsys_a.shape[0])

A = dtsys_a - eigval[0]*I
nullspace = linalg.null_space(A,rcond=1e-10)


u, s, vh = linalg.svd(A, full_matrices=True)
M, N = u.shape[0], vh.shape[1]
rcond = np.finfo(s.dtype).eps * max(M, N)
tol = np.amax(s, initial=0.) * rcond
num = np.sum(s > tol, dtype=int)
# num = 2
Q = vh[num:,:].T.conj()
print(f'tol is {tol}, s is {s}, nullspace is {nullspace}, Q is {Q}')

tol is 2.215657468664909e-14, s is [2.49460854e+01 2.26733286e+01 1.81332495e+01 2.63686619e-14], nullspace is [[-0.58743759-0.j]
 [-0.40298601-0.j]
 [-0.63204311-0.j]
 [-0.30502602-0.j]], Q is []


In [6]:
vh

array([[-0.79463859, -0.3925775 , -0.42910319, -0.17407722],
       [-0.04757392,  0.41532956,  0.07538039, -0.90529326],
       [ 0.57045623, -0.2334361 , -0.76150801, -0.20048147],
       [-0.20215322,  0.7866968 , -0.47988936,  0.33158472]])

In [ ]:
%timeit possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1e-2)


In [ ]:
if possible_states is not None:
    for ind in range(corresp_sensors.shape[0]):
        sensors = corresp_sensors[ind,:]
        state = possible_states[:,ind]
        print(f'Identified possible states:{state} for sensors {sensors}')

In [ ]:
import timeit

execution_time = timeit.timeit(lambda: ssr_solution.solve(error_bound = 1e-2), number=100)

print(execution_time)


0.36949859996093437


In [ ]:
att_dic

{0: [1, 3, 5, 6, 7], 1: [0, 2, 4]}

In [ ]:
print(np.finfo(np.double).precision)

15


In [ ]:
residuals_list = [1,2,3,4]
residual_min = min(residuals_list)
comb_list = residual_min<2*residual_min

In [ ]:
residuals_list[0]

1

In [2]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconstruct
from scipy import linalg
import numpy as np

TS = 0.02
s  = 0
Ac = np.matrix(
                [
                    # [0, 0, 1, 0],
                    # [0, 0, 0, 1],
                    # [0, 0, 0, 0],
                    # [0, 0, 0, 0]
                    [0, 1],  # x xdot
                    [0, 0],
                ])
Bc = np.matrix(
                [
                    # [0, 0],
                    # [0, 0],
                    # [1, 0],
                    # [0, 1]
                    [0],
                    [1],
                ]
            )
Cc = np.matrix(
                [
                    # [1, 0, 0, 0],
                    # [1, 0, 0, 0],
                    # [0, 1, 0, 0],
                    # [0, 1, 0, 0],
                    # [0, 0, 1, 0],
                    # [0, 0, 1, 0],
                    # [0, 0, 0, 1],
                    # [0, 0, 0, 1],
                    [1, 0],  # x1 xdot1 x2 xdot2
                    [0, 1],
                    [1, 0],
                    [0, 1],
                ]
            )

Dc = np.matrix(
                [
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0],
                    # [0, 0]
                    [0],
                    [0],
                    [0],
                    [0],
                ]
            )

dtsys_a, dtsys_b, dtsys_c, dtsys_d = SSProblem.convert_ct_to_dt(Ac,Bc,Cc,Dc,TS)
# define input output data
init_state1 = np.array([[1.],[2.]])
# u_seq = np.array([[1,1],[1,1],[1,1],[0,0]])
# assume sensors 1,3,5 are under attack
sensor_initial_states = [init_state1,init_state1,init_state1,init_state1]
# u_seq, tilde_y_his, noise_level = SSProblem.generate_attack_measurement(dtsys_a, dtsys_b, dtsys_c, dtsys_d,sensor_initial_states,
#                                                                         s = s,is_noisy = False, noise_level=0.001,u_seq = None)
u_seq = np.random.normal(3,2.5,size=(2,1))
tilde_y_his = np.random.normal(10,4,size=(2,4))

noise_level=0.001
ss_problem = SSProblem(dtsys_a, dtsys_b, dtsys_c, dtsys_d, tilde_y_his, 
                        attack_sensor_count=s,input_sequence=u_seq,measurement_noise_level= noise_level )
ssr_solution = SecureStateReconstruct(ss_problem)
possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 100)
print(f'u_seq:{u_seq}')
print(f'tilde_y_his: {tilde_y_his}')
print(f'possible_states: {possible_states}')




No possible state found. Consider relax the error bound


ValueError: need at least one array to concatenate

In [3]:
import numpy as np
a = np.zeros((2,3))
total_state = np.zeros(shape = a.shape)
total_state

array([[0., 0., 0.],
       [0., 0., 0.]])